The dataset used below is from the Medical Segmentation Decathlon whose brain tumour dataset is from the Multimodal Brain Tumour Segmentation Challenge (BRATS) 2016 and 2017. Link: http://medicaldecathlon.com/

# Install and import libraries

In [3]:
!pip install pascal-voc-writer
!pip install nibabel
!pip install pydicom

In [2]:
import os
import h5py
import tarfile
import gzip
import shutil

from pascal_voc_writer import Writer
import nibabel as nib
import pydicom

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image

from random import shuffle
import math
import re

# Mount drive

Only run below if running on Colab

In [2]:
#from google.colab import drive
#drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
#base_path = '/content/drive/My Drive/project_try_6/data'

# Access data

Only run below if running on local drive

In [1]:
#change directory containing data if necessary
base_path = 'D:\data' #external drive containing data

# Extract tar files

In [6]:
import tarfile
tar = tarfile.open(os.path.join(base_path, 'Task01_BrainTumour.tar'))
tar.extractall()
tar.close()

# Extract gzip files to get NIfTI files

In [4]:
base_folder = os.path.join(base_path, 'Task01_BrainTumour')
scan_folder = os.path.join(base_folder, 'imagesTr')
segments_folder = os.path.join(base_folder, 'labelsTr')
scan_save_folder = 'decathlon_scans'
segments_save_folder = 'decathlon_segments'

In [8]:
reg = re.compile('(BRATS_)[0-9]*(.nii.gz)')

In [10]:
count = 1

for f in os.listdir(scan_folder):
    if not reg.match(f):
        continue
    print(count)
    count += 1
    name = f.split('.')[0]
    with gzip.open(os.path.join(scan_folder, f), 'rb') as f_in:
        with open(os.path.join(scan_save_folder, name + '_T1.nii'), 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
    with gzip.open(os.path.join(segments_folder, f), 'rb') as f_in:
        with open(os.path.join(segments_save_folder, name + '_Segmentation.nii'), 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [11]:
for f in os.listdir(segments_save_folder):
    print(nib.load(os.path.join(segments_save_folder, f)).get_fdata().shape)

(240, 240, 155)
(240, 240, 155)
(240, 240, 155)
(240, 240, 155)
(240, 240, 155)
(240, 240, 155)
(240, 240, 155)
(240, 240, 155)
(240, 240, 155)
(240, 240, 155)
(240, 240, 155)
(240, 240, 155)
(240, 240, 155)
(240, 240, 155)
(240, 240, 155)
(240, 240, 155)
(240, 240, 155)
(240, 240, 155)
(240, 240, 155)
(240, 240, 155)
(240, 240, 155)
(240, 240, 155)
(240, 240, 155)
(240, 240, 155)
(240, 240, 155)
(240, 240, 155)
(240, 240, 155)
(240, 240, 155)
(240, 240, 155)
(240, 240, 155)
(240, 240, 155)
(240, 240, 155)
(240, 240, 155)
(240, 240, 155)
(240, 240, 155)
(240, 240, 155)
(240, 240, 155)
(240, 240, 155)
(240, 240, 155)
(240, 240, 155)
(240, 240, 155)
(240, 240, 155)
(240, 240, 155)
(240, 240, 155)
(240, 240, 155)
(240, 240, 155)
(240, 240, 155)
(240, 240, 155)
(240, 240, 155)
(240, 240, 155)
(240, 240, 155)
(240, 240, 155)
(240, 240, 155)
(240, 240, 155)
(240, 240, 155)
(240, 240, 155)
(240, 240, 155)
(240, 240, 155)
(240, 240, 155)
(240, 240, 155)
(240, 240, 155)
(240, 240, 155)
(240, 24

In [6]:
array = {}

# Obtain relevant slices from MRI scan volumes

In [11]:
scan_save_folder = os.path.join(base_path, scan_save_folder)
segments_save_folder = os.path.join(base_path, segments_save_folder)
for j in range(401, 485):
    subject = 'BRATS_{:03}'.format(j)
    scan_file_path = os.path.join(scan_save_folder, subject + '_T1.nii')
    segment = nib.load(os.path.join(segments_save_folder, subject + '_Segmentation.nii')).get_fdata()
    scan = nib.load(scan_file_path).get_fdata()
    slice_list = np.unique(np.nonzero(segment)[2])
    df = pd.DataFrame(data={'slice_index': [], 'area': []}, dtype=int)
    for x in slice_list:
        area = np.nonzero(scan[:, :, x, 1])[0].shape[0]
        df = df.append({'slice_index': x, 'area': area}, ignore_index=True)
    df.sort_values(by='area', inplace=True, ascending=False)
    roi = df.iloc[np.arange(0, 20)]
    for i in roi['slice_index'].values:
        tup = np.nonzero(segment[:, :, i])
        xs = tup[1]
        ys = tup[0]
        xmin = min(xs)
        ymin = min(ys)
        xmax = max(xs)
        ymax = max(ys)
        if (ymax - ymin) < 20 or (xmax - xmin) < 20 or tup[0].shape[0] < 400:
            continue
        print(subject + '-{}'.format(i))
        array[subject + '-{}'.format(i)] = (xmin, ymin, xmax, ymax)
        plt.imsave(os.path.join(os.path.join(base_path, 'decathlon_scans_images'), subject + '-{}.jpg'.format(i)), scan[:, :, i, 1], cmap='gray')

BRATS_401-75
BRATS_401-76
BRATS_401-74
BRATS_401-77
BRATS_401-73
BRATS_401-78
BRATS_401-72
BRATS_401-71
BRATS_401-79
BRATS_401-70
BRATS_401-69
BRATS_401-68
BRATS_401-67
BRATS_401-66
BRATS_402-75
BRATS_402-74
BRATS_402-76
BRATS_402-77
BRATS_402-73
BRATS_402-78
BRATS_402-72
BRATS_402-71
BRATS_402-79
BRATS_402-80
BRATS_402-70
BRATS_402-81
BRATS_402-82
BRATS_402-69
BRATS_402-83
BRATS_402-84
BRATS_402-85
BRATS_402-86
BRATS_402-68
BRATS_402-67
BRATS_403-78
BRATS_403-80
BRATS_403-77
BRATS_403-79
BRATS_403-81
BRATS_403-76
BRATS_403-82
BRATS_403-75
BRATS_403-83
BRATS_403-84
BRATS_403-74
BRATS_403-73
BRATS_403-85
BRATS_403-86
BRATS_403-72
BRATS_403-71
BRATS_403-87
BRATS_403-88
BRATS_403-70
BRATS_403-69
BRATS_404-78
BRATS_404-79
BRATS_404-77
BRATS_404-80
BRATS_404-81
BRATS_404-82
BRATS_404-83
BRATS_404-84
BRATS_404-76
BRATS_404-75
BRATS_404-85
BRATS_404-86
BRATS_404-74
BRATS_404-87
BRATS_404-73
BRATS_404-88
BRATS_404-89
BRATS_404-72
BRATS_404-90
BRATS_404-71
BRATS_405-80
BRATS_405-79
BRATS_405-76

BRATS_437-78
BRATS_437-77
BRATS_437-79
BRATS_437-80
BRATS_437-76
BRATS_437-75
BRATS_437-74
BRATS_437-81
BRATS_437-73
BRATS_437-72
BRATS_437-82
BRATS_437-71
BRATS_437-83
BRATS_437-70
BRATS_437-84
BRATS_437-69
BRATS_437-85
BRATS_437-68
BRATS_437-86
BRATS_437-67
BRATS_438-67
BRATS_438-66
BRATS_438-65
BRATS_438-64
BRATS_438-63
BRATS_438-62
BRATS_438-61
BRATS_438-60
BRATS_438-59
BRATS_438-58
BRATS_438-57
BRATS_438-56
BRATS_438-55
BRATS_439-78
BRATS_439-77
BRATS_439-76
BRATS_439-75
BRATS_439-79
BRATS_439-74
BRATS_439-73
BRATS_439-80
BRATS_439-72
BRATS_439-81
BRATS_439-71
BRATS_439-82
BRATS_439-70
BRATS_439-83
BRATS_439-69
BRATS_439-84
BRATS_439-68
BRATS_439-85
BRATS_439-67
BRATS_439-86
BRATS_440-80
BRATS_440-81
BRATS_440-82
BRATS_440-79
BRATS_440-78
BRATS_440-77
BRATS_440-83
BRATS_440-84
BRATS_440-76
BRATS_440-75
BRATS_440-85
BRATS_440-86
BRATS_440-74
BRATS_440-73
BRATS_440-87
BRATS_440-72
BRATS_440-88
BRATS_440-71
BRATS_440-89
BRATS_440-70
BRATS_441-75
BRATS_441-77
BRATS_441-76
BRATS_441-78

BRATS_473-78
BRATS_473-77
BRATS_473-76
BRATS_473-75
BRATS_473-74
BRATS_474-80
BRATS_474-79
BRATS_474-78
BRATS_474-77
BRATS_474-81
BRATS_474-75
BRATS_474-76
BRATS_474-82
BRATS_474-74
BRATS_474-73
BRATS_474-83
BRATS_474-72
BRATS_474-84
BRATS_474-71
BRATS_474-85
BRATS_474-86
BRATS_474-70
BRATS_474-87
BRATS_474-69
BRATS_474-88
BRATS_475-71
BRATS_475-70
BRATS_475-68
BRATS_475-69
BRATS_475-72
BRATS_475-67
BRATS_475-73
BRATS_475-74
BRATS_475-66
BRATS_475-75
BRATS_475-65
BRATS_475-76
BRATS_475-64
BRATS_475-77
BRATS_475-78
BRATS_475-63
BRATS_475-79
BRATS_475-80
BRATS_476-84
BRATS_476-85
BRATS_476-86
BRATS_476-87
BRATS_476-88
BRATS_476-89
BRATS_476-90
BRATS_476-91
BRATS_476-92
BRATS_476-93
BRATS_476-94
BRATS_476-95
BRATS_476-96
BRATS_476-97
BRATS_476-98
BRATS_476-99
BRATS_476-100
BRATS_476-101
BRATS_477-77
BRATS_477-76
BRATS_477-75
BRATS_477-71
BRATS_477-78
BRATS_477-73
BRATS_477-74
BRATS_477-79
BRATS_477-72
BRATS_477-80
BRATS_477-70
BRATS_477-69
BRATS_477-81
BRATS_477-82
BRATS_477-68
BRATS_477-

In [15]:
array

{'BRATS_001-71': (128, 74, 176, 154),
 'BRATS_001-73': (129, 72, 176, 152),
 'BRATS_001-74': (129, 71, 176, 151),
 'BRATS_001-72': (129, 73, 176, 153),
 'BRATS_001-75': (129, 71, 176, 151),
 'BRATS_001-69': (131, 72, 178, 155),
 'BRATS_001-70': (130, 73, 177, 154),
 'BRATS_001-76': (129, 71, 175, 150),
 'BRATS_001-77': (129, 71, 176, 150),
 'BRATS_001-68': (132, 71, 178, 155),
 'BRATS_001-78': (129, 70, 174, 150),
 'BRATS_001-67': (133, 72, 180, 155),
 'BRATS_001-79': (129, 70, 173, 150),
 'BRATS_001-80': (130, 69, 173, 150),
 'BRATS_001-81': (130, 68, 176, 149),
 'BRATS_001-66': (134, 73, 180, 157),
 'BRATS_001-82': (130, 68, 179, 148),
 'BRATS_001-83': (130, 68, 179, 147),
 'BRATS_001-65': (135, 74, 181, 158),
 'BRATS_001-84': (130, 69, 179, 142),
 'BRATS_002-72': (87, 50, 111, 90),
 'BRATS_002-73': (87, 50, 112, 89),
 'BRATS_002-74': (86, 50, 115, 90),
 'BRATS_002-75': (84, 50, 117, 91),
 'BRATS_002-76': (84, 48, 118, 91),
 'BRATS_002-77': (83, 47, 118, 92),
 'BRATS_002-78': (83, 47

# Save image filename and bounding box coordinates to csv

In [12]:
keys = list(array.keys())
df_out = pd.DataFrame({'name': [], 'xmin': [], 'ymin': [], 'xmax': [], 'ymax': []}, dtype=str)
for key in keys:
    xmin, ymin, xmax, ymax = array[key]
    df_out = df_out.append({'name': key, 'xmin': str(xmin), 'ymin': str(ymin), 'xmax': str(xmax), 'ymax': str(ymax)}, 
                           ignore_index=True)
df_out.to_csv(os.path.join(base_path, 'decathlon_labels_and_bbox.csv'), index=False)